In [4]:
# read jsonl

import json
from datasets import load_dataset
from verl.utils.reward_score.prime_math import compute_score
# from verl.workers.reward_manager import PrimeRewardManager

In [5]:
from verl.utils.reward_score.prime_math import compute_score, match_answer, grade_answer

In [6]:

data = []
for node_id in range(64):
    for chunk_id in range(4):
        with open(f'../sgl_inference_results/big-math-rl-verified_problems/node_{node_id}/batch_results_chunk_{chunk_id}.jsonl', 'r') as f:
            for id, line in enumerate(f):
                if id >= 64:
                    break
                data.append(json.loads(line))
d_ = [(int(d['custom_id'].split('-')[1]), d['response']['body']['choices']['message']['content']) for d in data]

In [7]:
from datasets import load_dataset
dataset = load_dataset('SynthLabsAI/Big-Math-RL-Verified', split='train')

In [4]:

from tqdm import tqdm
saved_data = {}
for d in tqdm(d_):
    if d[0] not in saved_data:
        saved_data[d[0]] = {
            "question": dataset['problem'][d[0]],
            "answer": dataset['answer'][d[0]],
            "model_outputs": [d[1]]
        }
    else:
        saved_data[d[0]]['model_outputs'].append(d[1])

100%|██████████| 16384/16384 [02:23<00:00, 114.33it/s]


In [8]:
saved_data = json.load(open('sampled_data.json', 'r'))

In [9]:
# json.dump(saved_data, open('sampled_data.json', 'w'))

In [10]:
saved_data[0]['model_outputs']

KeyError: 0

In [44]:
import requests

url = f"http://localhost:30000/v1/chat/completions"

input_template = """You are a teacher and your task is to grade the student's answer with the reference answer.

Question: [QUESTION]

Student's Answer: [STUDENT'S ANSWER]

Reference Answer: [REFERENCE ANSWER]

You only need to refer to the reference answer to grade the student's answer. Sometimes the student's answer is expressed in a different way from the reference answer, but the meaning is the same, and you should still consider it correct. If they are not equivalent in mathematical sense, you should consider it incorrect.

Note 1: You don't need to solve the problem yourself. Just grade the student's answer based on the reference answer.

Note 2: If the reference answer is a range, please make sure the student's answer is strictly identical, including the open or closed interval.

Note 3: If the reference answer is an expression and it looks like the student's answer is equivalent to the reference answer, you should present the derivation process to check if they are equivalent.

Note 4: If the reference answer includes multiple solutions, please make sure the student's answer covers all of them.

Please provide a brief explanation (a few sentences) of your grading process and put your final grade in the following format:

Final Grade: CORRECT or INCORRECT
"""

In [45]:
q = "Solve the inequality system: $\\left\{\\begin{array}{c}3x-1>x+1\\ \\frac{4x-5}{3}\\lleq x\end{array}l\\right.$."
ref = "1 < x \\leq 5"
answer = "(1, 5)"
inputs = input_template.replace("[QUESTION]", q).replace("[STUDENT'S ANSWER]", answer).replace("[REFERENCE ANSWER]", ref)
# print(inputs)
data = {
    "model": "Qwen/Qwen2.5-32B-Instruct",
    "messages": [{"role": "user", "content": inputs}],
}
response = requests.post(url, json=data)
print(not "INCORRECT" in response.json()['choices'][0]['message']['content'])

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
/tmp/ipykernel_2240297/189168276.py:1: SyntaxWarning: invalid escape sequence '\{'
  q = "Solve the inequality system: $\\left\{\\begin{array}{c}3x-1>x+1\\ \\frac{4x-5}{3}\\lleq x\end{array}l\\right.$."


False


In [46]:
print(response.json()['choices'][0]['message']['content'])

The student's answer is (1, 5), which represents an open interval from 1 to 5. The reference answer is \(1 < x \leq 5\), which represents an interval where \(x\) is greater than 1 and less than or equal to 5. The discrepancy lies in the inclusion of the endpoint 5. The student's answer excludes the endpoint 5 (an open interval), whereas the reference answer includes it (a half-open interval). Therefore, the student's answer does not fully match the reference answer.

Final Grade: INCORRECT


In [47]:
# // ... existing code ...

from multiprocessing import Pool, Manager
import os
from functools import partial
import time
from tqdm import tqdm

def process_single_response(question_id, model_output, question, ground_truth, input_template, url):
    model_match_answer = match_answer(model_output)
    
    if model_match_answer[0]:
        extracted_model_output = model_match_answer[1]
    else:
        extracted_model_output = ""
    
    is_correct = False
    judge_comment = ""
    # grade simple algebra questions
    if grade_answer(extracted_model_output, ground_truth):
        is_correct = True
    
    
    
    else:
        try:
            if "\\pi" in extracted_model_output or "\\pi" in ground_truth:
                equivs = []
                for pi in [math.pi, 3.14]:
                    equivs.append(math_equal(extracted_model_output, ground_truth, timeout=True, pi=pi))
                is_correct = any(equivs)
            else:
                is_correct = math_equal(extracted_model_output, ground_truth, timeout=True)
        except:
            is_correct = False
    
    llm_is_correct = None
    if not is_correct and model_match_answer[0]:
        # use LLM to grade the answer
        inputs = input_template.replace("[QUESTION]", question).replace("[STUDENT'S ANSWER]", extracted_model_output).replace("[REFERENCE ANSWER]", ground_truth)
        data = {
            "model": "Qwen/Qwen2.5-7B-Instruct",
            "messages": [{"role": "user", "content": inputs}],
        }
        try:
            response = requests.post(url, json=data)
            llm_is_correct = not "INCORRECT" in response.json()['choices'][0]['message']['content']
            judge_comment = response.json()['choices'][0]['message']['content']
            # Add small delay to avoid overwhelming the API
        except Exception as e:
            print(f"Error in LLM API call for question {question_id}: {e}")
    
    return {
        'is_correct': (is_correct, llm_is_correct, judge_comment),
        'extracted_output': extracted_model_output,
    }

def process_question(args):
    question_id, question_data, input_template, url = args
    results = []
    
    for idx, model_output in enumerate(question_data['model_outputs']):
        result = process_single_response(
            question_id,
            model_output,
            question_data['question'],
            question_data['answer'],
            input_template,
            url
        )
        results.append(result)
    
    return {
        'question_id': question_id,
        'question': question_data['question'],
        'ground_truth': question_data['answer'],
        'results': results
    }

# Get the number of CPU cores and use 75% of them
num_cores = max(1, int(0.75 * os.cpu_count()))

# Prepare the arguments for parallel processing
args_list = [
    (qid, qdata, input_template, url)
    for qid, qdata in saved_data.items()
]

# Process questions in parallel
all_results = {}
with Pool(num_cores) as pool:
    for result in tqdm(pool.imap(process_question, args_list), 
                      total=len(args_list), 
                      desc="Processing questions"):
        qid = result['question_id']
        all_results[qid] = result
        
        # Calculate statistics for this question
        math_correct = sum(1 for r in result['results'] if r['is_correct'][0])
        llm_correct = sum(1 for r in result['results'] if r['is_correct'][1])
        total_responses = len(result['results'])
        
        print(f"\nQuestion {qid}:")
        print(f"Math correct: {math_correct}/{total_responses} ({math_correct/total_responses*100:.2f}%)")
        print(f"LLM correct: {llm_correct}/{total_responses} ({llm_correct/total_responses*100:.2f}%)")

# Save results to file
with open('grading_results_qwen_32b_instruct.json', 'w') as f:
    json.dump(all_results, f)

# Final summary across all questions
total_math_correct = sum(
    sum(1 for r in qdata['results'] if r['is_correct'][0])
    for qdata in all_results.values()
)
total_llm_correct = sum(
    sum(1 for r in qdata['results'] if r['is_correct'][1])
    for qdata in all_results.values()
)
total_responses = sum(len(qdata['results']) for qdata in all_results.values())

print("\nOverall Summary:")
print(f"Total math correct: {total_math_correct}/{total_responses} ({total_math_correct/total_responses*100:.2f}%)")
print(f"Total LLM correct: {total_llm_correct}/{total_responses} ({total_llm_correct/total_responses*100:.2f}%)")
# // ... existing code ...

Processing questions:   0%|          | 1/512 [01:15<10:42:22, 75.43s/it]


Question 0:
Math correct: 9/32 (28.12%)
LLM correct: 0/32 (0.00%)

Question 1:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 1024:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 1025:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 2048:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 2049:
Math correct: 0/32 (0.00%)
LLM correct: 28/32 (87.50%)


Processing questions:   1%|▏         | 7/512 [01:23<1:16:17,  9.07s/it] 


Question 3072:
Math correct: 0/32 (0.00%)
LLM correct: 32/32 (100.00%)

Question 3073:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 3924:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 3925:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 4948:
Math correct: 14/32 (43.75%)
LLM correct: 0/32 (0.00%)


Processing questions:   2%|▏         | 12/512 [01:34<46:19,  5.56s/it] 

...

Processing questions:  20%|█▉        | 101/512 [02:56<09:31,  1.39s/it]


Question 49136:
Math correct: 0/32 (0.00%)
LLM correct: 30/32 (93.75%)

Question 49137:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 50160:
Math correct: 0/32 (0.00%)
LLM correct: 31/32 (96.88%)

Question 50161:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 51012:
Math correct: 0/32 (0.00%)
LLM correct: 0/32 (0.00%)

Question 51013:
Math correct: 31/32 (96.88%)
LLM correct: 0/32 (0.00%)

Question 52036:
Math correct: 0/32 (0.00%)
LLM correct: 0/32 (0.00%)

Question 52037:
Math correct: 31/32 (96.88%)
LLM correct: 1/32 (3.12%)

Question 53060:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 53061:
Math correct: 31/32 (96.88%)
LLM correct: 0/32 (0.00%)

Question 54084:
Math correct: 26/32 (81.25%)
LLM correct: 2/32 (6.25%)

Question 54085:
Math correct: 8/32 (25.00%)
LLM correct: 24/32 (75.00%)

Question 54936:
Math correct: 32/32 (100.00%)
LLM correct: 0/32 (0.00%)

Question 54937:
Math correct: 0/32 (0.00%)
LLM correct: 0/32 (

Processing questions: 100%|██████████| 512/512 [03:33<00:00,  2.39it/s]


Question 156961:
Math correct: 0/32 (0.00%)
LLM correct: 0/32 (0.00%)

Question 157984:
Math correct: 28/32 (87.50%)
LLM correct: 0/32 (0.00%)

Question 157985:
Math correct: 10/32 (31.25%)
LLM correct: 0/32 (0.00%)

Question 159008:
Math correct: 2/32 (6.25%)
LLM correct: 0/32 (0.00%)

Question 159009:
Math correct: 1/32 (3.12%)
LLM correct: 2/32 (6.25%)

Question 160032:
Math correct: 26/32 (81.25%)
LLM correct: 0/32 (0.00%)

Question 160033:
Math correct: 11/32 (34.38%)
LLM correct: 21/32 (65.62%)

Question 160884:
Math correct: 31/32 (96.88%)
LLM correct: 0/32 (0.00%)

Question 160885:
Math correct: 26/32 (81.25%)
LLM correct: 4/32 (12.50%)

Question 161908:
Math correct: 26/32 (81.25%)
LLM correct: 0/32 (0.00%)

Question 161909:
Math correct: 5/32 (15.62%)
LLM correct: 0/32 (0.00%)

Question 162932:
Math correct: 19/32 (59.38%)
LLM correct: 10/32 (31.25%)

Question 162933:
Math correct: 25/32 (78.12%)
LLM correct: 0/32 (0.00%)

Question 163956:
Math correct: 0/32 (0.00%)
LLM corr


Overall Summary:
Total math correct: 11636/16384 (71.02%)
Total LLM correct: 1203/16384 (7.34%)


In [42]:

print("\nOverall Summary:")
print(f"Total math correct: {total_math_correct}/{total_responses} ({total_math_correct/total_responses*100:.2f}%)")
print(f"Total LLM correct: {total_llm_correct}/{total_responses} ({total_llm_correct/total_responses*100:.2f}%)")
# // ... existing code ...


Overall Summary:
Total math correct: 11636/16384 (71.02%)
Total LLM correct: 1328/16384 (8.11%)


In [43]:
# collect all the response that math incorrect and LLM correct
# all results

for k, v in all_results.items():

    for r in v['results']:
        if not r['is_correct'][0] and r['is_correct'][1]:
            print(v['question'])
            print("model output:", r['extracted_output'])
            print("-"*10)
            print("ground truth:", v['ground_truth'])
            print("-"*10)
            print("judge comment:", r['is_correct'][2])

Given that one of the symmetry centers of the function $f(x)=2\cos ( \frac {π}{3}x+φ)$ is $(2,0)$, and $φ∈(0,π)$, find the value of $φ$.
model output: \dfrac{5\pi}{6}
----------
ground truth: φ= \frac {5π}{6}
----------
judge comment: The student's answer is \(\dfrac{5\pi}{6}\) and the reference answer is \(\phi = \dfrac{5\pi}{6}\). Both answers are mathematically identical. The notation used by the student omits the phrase "phi equals," but the value provided matches the reference answer exactly.

Final Grade: CORRECT
Given that one of the symmetry centers of the function $f(x)=2\cos ( \frac {π}{3}x+φ)$ is $(2,0)$, and $φ∈(0,π)$, find the value of $φ$.
model output: \dfrac{5\pi}{6}
----------
ground truth: φ= \frac {5π}{6}
----------
judge comment: The student's answer is \(\dfrac{5\pi}{6}\), which is mathematically identical to the reference answer \(φ= \frac {5π}{6}\). There is no difference in the expression or the value. Therefore, the student's answer is correct.

Final Grade: CO